In [1]:
import numpy as np
import pandas as pd
from reformat import *
from architecture.WordSegPreProcessing import *

fn = "train.tsv"
x, y = file_to_table(read_file(fn))


## Prepping Data

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
preprocessor = WordSegPreProcessing(X_train, y_train, segment_to_tag)

In [3]:
X_train, y_train = preprocessor.x, preprocessor.y
X_test, y_test = preprocessor.extract_pairs(X_test, y_test)


## Supervised NGramTagger with Backoff

In [4]:
from architecture.NgramSupervisedTagger import NGramSupervisedTagger
ng = NGramSupervisedTagger(X_train, y_train, ngram_choice=2)
ng.create_n_gram_tagger()
ng.f1_by_tags(X_test, y_test)["I"] #F1 score for just I tags

/Users/a8nguyen/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/a8nguyen/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details

0.9065628476084538

## Supervised HMM

In [5]:
from architecture.HMMSupervisedTagger import HMMSupervisedTagger
hmm = HMMSupervisedTagger(X_train, y_train)
hmm.train()
hmm.f1_by_tags(X_test, y_test)["I"]


0.89321468298109

The same thing as HMM Tagger but instead of just using the provided characters, I manually engineered some features (probably not the best bleh)... You can check it out in `architecture.WordSegPreProcessing.apply_features`

In [6]:
from architecture.HMMSupervisedTagger import HMMSupervisedTagger
hmm2 = HMMSupervisedTagger(preprocessor.generate_features(X_train), y_train)
hmm2.train()
hmm2.f1_by_tags(preprocessor.generate_features(X_test), y_test)["I"]


0.9547856723429242

## Unsupervised

In [7]:
from architecture.utils import *
x, y = file_to_table(read_file(fn))
feedX, feedY = preprocessor.extract_pairs(x[:650], y[:650])


In [8]:
preprocessor.index_tag

{'I': 0, 'B': 1}

In [9]:
feedX = list(map(lambda x: preprocessor.let2index(x), feedX ))
feedY = list(map(lambda y: preprocessor.tag2index(y), feedY))


In [10]:
#initialize with some probs
import nltk
import numpy as np
init_tmat = np.zeros((len(preprocessor.index_tag),
                     len(preprocessor.index_tag)))

init_emission = np.zeros(
    (len(preprocessor.index_tag), len(preprocessor.index_vocab)))
trainer = nltk.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(
           tuple_xy4nltk(feedX, feedY))
for k in tagger._transitions.keys():
    for v in tagger._transitions[k].samples():
        init_tmat[k][v] = tagger._transitions[k].prob(v)
    for let in tagger._outputs[k].samples():
        init_emission[k][let] = tagger._outputs[k].prob(let)
init_state_distrib = np.array([0, 1.0])  # obvi because always start with B



In [13]:
unsupervised = UnSupervised(init_tmat, init_emission, init_state_distrib) # obvi because always start with B)

In [14]:
unsupervised.calculate_beta_forward(feedX[33])

array([[0.00000000e+00, 6.99325629e-05, 0.00000000e+00, 1.32803428e-06,
        0.00000000e+00, 0.00000000e+00],
       [7.81059312e-03, 6.99325629e-05, 1.66540159e-05, 1.81527761e-06,
        1.69314090e-07, 5.43416174e-09]])

In [15]:
unsupervised.calculate_alpha_backward(feedX[33])


[[0.        0.        0.        0.        0.        0.       ]
 [0.        0.        0.        0.        0.        0.0039053]]


array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.0039053]])

In [12]:
import nltk
from architecture.utils import *
import numpy as np


class UnSupervised:
    def __init__(self, tmat, emission, initial_distrib):

        self.tmat = tmat
        self.emission = emission
        self.initial_prob = initial_distrib

        self.num_states = len(self.tmat)
        self.num_letters = len(self.emission[0])

    def maximize(self):  # viterbi
        return

    def decode(self):  # tagging the input
        return

    def estimate(self, o_seq):  # forward backward algorithm
        backward_table = np.zeros((self.num_states, len(o_seq)))  # vit table
        for i in range(len(o_seq)):  # Filling in the table
            prev_prob = tf.reshape(self.forward[i, :], [1, -1])

    def calculate_alpha_backward(self, o_seq, end_prob=[0.5, 0.5]):
        backward_table = np.zeros((self.num_states, len(o_seq)))  # vit table
        #backward_tn = [self.emission[i][o_seq[0]]
        #               for i in range(self.num_states)]
        
        backward_table[:, -1] = np.multiply(backward_tn, end_prob)
        print(backward_table)

        for i in range(0, len(o_seq)-1, -1):  # start filling in the table
            beta_s_t = 0
            for tag_cell in range(self.num_states):
                for tag_prev in range(self.num_states):
                    beta_s_t += backward_table[tag_prev][i+1] * \
                        self.tmat[tag_prev][tag_cell] * \
                        self.emission[tag_cell][o_seq[i]]
                backward_table[tag_cell][i] = beta_s_t
        return backward_table

    def calculate_beta_forward(self, o_seq):

        forward_table = np.zeros((self.num_states, len(o_seq)))  # vit table
        emission_t0 = [self.emission[i][o_seq[0]]
                       for i in range(self.num_states)]
        forward_table[:, 0] = np.multiply(emission_t0, self.initial_prob)

        for i in range(1, len(o_seq)):  # start filling in the table
            alpha_s_t = 0
            for tag_cell in range(self.num_states):
                for tag_prev in range(self.num_states):
                    alpha_s_t += forward_table[tag_prev][i-1] * \
                        self.tmat[tag_prev][tag_cell] * \
                        self.emission[tag_cell][o_seq[i]]
                forward_table[tag_cell][i] = alpha_s_t
        return forward_table
